# Purposes of this document

the goal of this jupyter is to run the BMIloop in steps  
we can  
1. understand what is happening. 
2. develop new components  

along these reasons,  
we will clearly point out where the components are  
we focus on how information is transmitted between components  

note all of this is in the branch sijia as of 2021 Jan


# setting up the simulation components

this section largely copyied and pasted from   
bmi3d-sijia(branch)-bulti_in_experiemnts
https://github.com/sijia66/brain-python-interface/blob/master/built_in_tasks/sim_task_KF.py

In [1]:
# import libraries
# make sure these directories are in the python path., 
from bmimultitasks import SimBMIControlMulti, SimBMICosEncKFDec, BMIControlMultiNoWindow
from features import SaveHDF
from features.simulation_features import get_enc_setup, SimKFDecoderRandom, SimCosineTunedEnc,SimIntentionLQRController, SimClockTick
from features.simulation_features import SimHDF, SimTime

from riglib import experiment

from riglib.stereo_opengl.window import FakeWindow

import time
import numpy as np

/home/sijia-aw/BMi3D_my/lab_bmi3d/riglib/blackrock/cerelink.py:13: UserWarning: Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system
  warnings.warn("Unable to import cerebus library. Check if is installed if using the Blackrock NeuroPort system")


##  set up trial information

In [2]:
#generate task params
N_TARGETS = 8
N_TRIALS = 1
seq = SimBMIControlMulti.sim_target_seq_generator_multi(
N_TARGETS, N_TRIALS)

##  simulation encoder decoder setup

In [3]:
#clda on random 
DECODER_MODE = 'trainedKF' # in this case we load simulation_features.SimKFDecoderRandom
ENCODER_TYPE = 'cosine_tuned_encoder'
LEARNER_TYPE = 'dumb' # to dumb or not dumb it is a question 'feedback'
UPDATER_TYPE = 'none' #none or "smooth_batch"


SAVE_HDF = False

SAVE_SIM_HDF = True #this makes the task data available as exp.task_data_hist

DEBUG_FEATURE = False

#neuron set up : 'std (20 neurons)' or 'toy (4 neurons)' 
N_NEURONS, N_STATES, sim_C = get_enc_setup(sim_mode = 'toy')

features.simulation_features: get_enc_setup has a tuning_level of 1 



## from the setup options, set up experiment

In [4]:
# set up assist level
assist_level = (0, 0)

#base_class = SimBMIControlMulti
base_class = BMIControlMultiNoWindow
feats = []

#set up intention feedbackcontroller
#this ideally set before the encoder
feats.append(SimIntentionLQRController)

#set up the encoder
if ENCODER_TYPE == 'cosine_tuned_encoder' :
    feats.append(SimCosineTunedEnc)
    print(f'{__name__}: selected SimCosineTunedEnc\n')
    
    
   #take care the decoder setup
if DECODER_MODE == 'random':
    feats.append(SimKFDecoderRandom)
    print(f'{__name__}: set base class ')
    print(f'{__name__}: selected SimKFDecoderRandom \n')
else: #defaul to a cosEnc and a pre-traind KF DEC
    from features.simulation_features import SimKFDecoderSup
    feats.append(SimKFDecoderSup)
    print(f'{__name__}: set decoder to SimKFDecoderSup\n')
    
    
#you know what? 
#learner only collects firing rates labeled with estimated estimates
#we would also need to use the labeled data
#to update the decoder.
if UPDATER_TYPE == 'smooth_batch':
    from features.simulation_features import SimSmoothBatch
    feats.append(SimSmoothBatch)
else: #defaut to none 
    print(f'{__name__}: need to specify an updater')
    
if DEBUG_FEATURE: 
    from features.simulation_features import DebugFeature
    feats.append(DebugFeature)
    
if SAVE_HDF: feats.append(SaveHDF)
if SAVE_SIM_HDF: feats.append(SimHDF)
    
    
#pass the real time limit on clock
feats.append(SimClockTick)
feats.append(SimTime)


#save everthing in a kw
kwargs = dict()
kwargs['sim_C'] = sim_C

__main__: selected SimCosineTunedEnc

__main__: set decoder to SimKFDecoderSup

__main__: need to specify an updater


## make our experiment class

In [5]:
#spawn the task
Exp = experiment.make(base_class, feats=feats)
#print(Exp)
exp = Exp(seq, **kwargs)
exp.init()

metaclass constructor
(<class 'features.simulation_features.SimIntentionLQRController'>, <class 'features.simulation_features.SimCosineTunedEnc'>, <class 'features.simulation_features.SimKFDecoderSup'>, <class 'features.simulation_features.SimHDF'>, <class 'features.simulation_features.SimClockTick'>, <class 'features.simulation_features.SimTime'>, <class 'bmimultitasks.BMIControlMultiNoWindow'>)
[<class 'features.simulation_features.SimIntentionLQRController'>, <class 'features.simulation_features.SimCosineTunedEnc'>, <class 'features.simulation_features.SimKFDecoderSup'>, <class 'features.simulation_features.SimHDF'>, <class 'features.simulation_features.SimClockTick'>, <class 'features.simulation_features.SimTime'>]

features.simulation_features.SimIntentionLQRController: LQRController used 

0.0
0.0
finished executing Experiment.__init__

SimCosineTunedEnc SSM: State space: ['hand_px', 'hand_py', 'hand_pz', 'hand_vx', 'hand_vy', 'hand_vz', 'offset'] 

neural encoder init function  

## test run a bit

print(exp.state)
exp.run() 

# now comes to the step through BMIloop


## decode neural features to move cursor
this section basically steps through the move_plant stuff

@riglib.bmi.BMILoop.move_plant


### bmi feature extraction, eh
#riglib.bmi: 1202
feature_data = exp.get_features()
feature_data

###  Determine the target_state and save to file
current_assist_level = exp.get_current_assist_level()
if np.any(current_assist_level > 0) or exp.learn_flag:
    target_state = exp.get_target_BMI_state(self.decoder.states)
else:
    target_state = np.ones([exp.decoder.n_states, exp.decoder.n_subbins]) * np.nan


#decode the new features
#riglib.bmi.bmiloop: line 1245
neural_features = feature_data[exp.extractor.feature_type]
tmp = exp.call_decoder(neural_features, target_state, **kwargs)

#saved as task data
exp.task_data['internal_decoder_state'] = tmp
tmp

#### reset the plant position
####  @riglib.bmi.BMILoop.move_plant  line:1254
exp.plant.drive(exp.decoder)
exp.plant.position

# assemble into a complete loop

In [6]:
#riglib.experiment: line 597 - 601
#exp.next_trial = next(exp.gen)
#-+exp._parse_next_trial()


# we need to set the initial state
# per fsm.run:  line 138


## Initialize the FSM before the loop
exp.set_state(exp.state)

while exp.state is not None:
    if exp.debug: 
        # allow ungraceful termination if in debugging mode so that pdb 
        # can catch the exception in the appropriate place
        exp.fsm_tick()
    else:
        # in "production" mode (not debugging), try to capture & log errors gracefully
        try:
            exp.fsm_tick()
        except:
            print('failed')
            exp.print_to_terminal("Error in FSM tick")
            exp.state = None
            exp.terminated_in_error = True

            exp.termination_err = io.StringIO()
            traceback.print_exc(None, exp.termination_err)
            exp.termination_err.seek(0)

            exp.print_to_terminal(exp.termination_err.read())
            exp.termination_err.seek(0)
            
if exp.verbose: print("end of FSM.run, task state is", exp.state)


0.0
runseq
0.0
0.0
assist_level at final value after 0 successful trials
end of FSM.run, task state is None


In [7]:
exp.chain_length

2

# Analyze HDF file

In [8]:
len(exp.task_data_hist)

101

In [9]:
exp.state_log

[('wait', 0.0),
 ('target', 0.016666666666666666),
 ('hold', 0.03333333333333333),
 ('targ_transition', 0.23333333333333334),
 ('target', 0.25),
 ('hold', 1.2),
 ('targ_transition', 1.4166666666666667),
 ('reward', 1.4333333333333333),
 ('wait', 1.65),
 ('target', 1.6666666666666667),
 (None, 1.6833333333333333)]